In [ ]:
'''User Parameters'''
RETENTION_TIME = 6 #hours
PIREP_LYR_ID = "<GUID FOR PIREP LAYER>"
UPDATE_TIME = 1 #hours

In [ ]:
import requests, json
from datetime import datetime as dt, timedelta
from arcgis.gis import GIS
from arcgis.geometry import Point
from arcgis.features import Feature

In [ ]:
gis = GIS('home')

In [ ]:
'''Query URL to Aviation Weather BETA'''
url = r'https://beta.aviationweather.gov/cgi-bin/data/pirep.php?&format=decoded&age={}'.format(UPDATE_TIME)

In [ ]:
'''Retrive PIREPs Layer'''
pirep_lyr = gis.content.get(PIREP_LYR_ID).layers[0]
pirep_lyr

In [ ]:
'''Clean Old PIREPs by timestamp'''
retain_time = dt.now() - timedelta(hours=RETENTION_TIME)
query = "dtg < TIMESTAMP '{}'".format(retain_time.strftime('%Y-%m-%d %H:%M:%S'))
result = pirep_lyr.delete_features(where=query)
for f in result['deleteResults']:
    if f['success'] is False:
        print(f)

In [ ]:
class PIREP():
    '''Class for handling manipulation of PIREP info'''
    def __init__(self):
        self.type = None
        self.priority = None
        self.location = None
        self.dtg = None
        self.retrievedtime = None
        self.flightlevel = None
        self.aircrafttype = None
        self.skycover = None
        self.temperature = None
        self.winddirection = None
        self.windspeed = None
        self.turbulence = None
        self.icing = None
        self.remarks = None
        self.weather = None
        self.visibility = None
        self.text = None
        
    @staticmethod
    def _splitLine(line):
        '''splits text formated line at : '''
        try:
            return line.split(': ')[1].strip()
        except:
            return None
    
    @staticmethod
    def _getRemarks(line):
        '''Returns plain text of Remarks or None if no Remarks'''
        splt = line.split('/')
        for l in splt:
            if l[:2] == 'RM':
                return l[3:]
        return None
    
    @staticmethod
    def _parseLocation(line):
        '''Returns lat,lon as comma delimited pair'''
        loc = PIREP._splitLine(line)
        loc = loc.replace('latitude','')\
                .replace('longitude','')\
                .replace(' ','')
        return loc
        
    @staticmethod
    def _parseFlightLevel(line):
        '''Parses Flight Level and converts to numerical feet'''
        fl = PIREP._splitLine(line)
        fl = fl.split(' ')[0]
        return int(fl)*100
    
    @staticmethod
    def _parseDTG(line):
        '''parses message time and return DateTime object'''
        dtg = line.split(' at ')[1]
        return dt.strptime(dtg,'%H%M %Z %d %b %Y')

    @staticmethod
    def _parseTemp(line):
        '''Parses temperature and returns a numerical value in celsium'''
        temp = PIREP._splitLine(line)
        return int(temp[:-1])
    
    def _parseWinds(self,line):
        '''
        Parses wind vector and updates direction and speed 
        attributes as numerical values
        '''
        wnd = self._splitLine(line)
        if not wnd is None:
            splt = wnd.split(' ')
            self.winddirection = int(splt[0])
            spd = splt[2][:-3]
            self.windspeed = int(spd)
    
    def _getPriority(self):
        '''Converts PIREP priority to plain text'''
        if self.type == 'PIREP':
            header = self.text.split('/')[0]
            pri = header.split(' ')[1].strip()
            if pri == 'UA':
                self.priority = 'ROUTINE'
            else:
                self.priority = 'URGENT'
        else:
            self.priority = None       
        
    @classmethod
    def parseReport(cls,lines,retreived_time):
        '''class method to convert list of PIREP lines into PIREP Object'''
        data = cls()
        data.retrievedtime = retreived_time
        data.text = lines[0].split(': ')[1]
        data.remarks = data._getRemarks(data.text)
        data.type = lines[0][:5]
        data._getPriority()
        
        for line in lines[1:]:
            line = line.strip()
            if line.startswith('Observed'):
                data.dtg = data._parseDTG(line)
            elif line.startswith('Aircraft'):
                data.aircrafttype = data._splitLine(line)
            elif line.startswith('Location'):
                data.location = data._parseLocation(line)
            elif line.startswith('Flight level'):
                data.flightlevel = data._parseFlightLevel(line)
            elif line.startswith('Turbulence'):
                data.turbulence = data._splitLine(line)
            elif line.startswith('Skies'):
                data.skycover = data._splitLine(line)
            elif line.startswith('Temperature'):
                data.temperature = data._parseTemp(line)
            elif line.startswith('Winds'):
                data._parseWinds(line)
            elif line.startswith('Icing'):
                data.icing = data._splitLine(line)
            elif line.startswith('Weather'):
                data.weather = data._splitLine(line)
            elif line.startswith('Visbility'):
                data.visibility = data._splitLine(line)
        return data
    
    def toDict(self):
        return self.__dict__
    
    def getGeometry(self):
        if not self.location is None:
            lat,lon = self.location.split(',')
            pt = Point({"x":lon, "y":lat, "spatialReference":{"wkid" : 4326}})
            return pt
        else:
            return None
    
    def toFeature(self):
        pt = self.getGeometry()
        attr = self.toDict()
        if not pt is None:
            geom = json.loads(pt.JSON)
        else:
            geom = None
        return Feature(geom,attr)
        

In [ ]:
'''get PIREP data'''
received = dt.now() #stored retrieved time for data custody
data = requests.get(url)
lines = data.text.split('\n') #split response by line break

In [ ]:
'''Group lines into all lines for one message'''
messages = []
message_lines = []
for line in lines:
    if line[:10] in ['PIREP Text','AIREP Text']: #check for both PIREP and AIREPs
        if len(message_lines): messages.append(message_lines)
        message_lines = [line]
    else:
        message_lines.append(line)

In [ ]:
'''Parse messages and update feature layer'''
pireps = []
for m in messages:
    p = PIREP.parseReport(m, received)
    try:
        pirep_lyr.edit_features(adds=[p.toFeature()])
    except Exception as e:
        print(str(e))